This notebook plots ground truth bounding boxes imported via the network architecture, at least using the custom Python data handling layers.

In [ ]:
import os
import sys
import pprint

import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pyplot as plt
import time

In [ ]:
def add_path(path):
    if path not in sys.path:
        sys.path.insert(0, path)
        
add_path('/home/ubuntu/src/py-faster-rcnn-windowed_input/caffe-fast-rcnn/python')
add_path('/home/ubuntu/src/py-faster-rcnn-windowed_input/lib')
import caffe
from datasets.factory import get_imdb, list_imdbs
from fast_rcnn.test import test_net, plot_all_bboxes
from fast_rcnn.train import get_training_roidb, train_net, SolverWrapper
from fast_rcnn.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
import roi_data_layer.minibatch as minibatch
print "Loaded caffe version {:s} from {:s}.".format(caffe.__version__, caffe.__path__[0])

In [ ]:
class_names = {0: 'buildings', 
               1: 'misc. manmade structures',
               2: 'road',
               3: 'track',
               4: 'trees',
               5: 'crops',
               6: 'waterway',
               7: 'standing water',
               8: 'vehicle large',
               9: 'vehicle small'}
def get_color(class_name):
    import colorsys
    h = (hash(class_name) % np.pi) / np.pi
    # v = (hash(class_name) % 10)/20. + .5
    N_v = 3
    v = .5/(N_v-1)*np.floor((hash(class_name) % (N_v*np.pi))/np.pi) + .5
    return colorsys.hsv_to_rgb(h, .8, v)

In [ ]:
# configure plotting
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['figure.figsize'] = (15, 15)

caffe.set_mode_gpu()

In [ ]:
N_classes = len(class_names) + 1
classes = (0, 1, 4, 7, 8, 9)
arch = 'VGG16'
appendix = '' # This will codify which classes to train (if not all, in which case this string should be empty)
infix = '.' # for directories
if classes is not None:
    appendix = '_'+''.join(['{:d}'.format(c) for c in classes])
    infix = '{:d}'.format(len(classes))+'_classes'
    N_classes = len(classes) + 1

print infix, appendix

In [ ]:
this_dir = os.getcwd()
model_dir = os.path.join(this_dir, '..', 'models', arch, infix)
solver_file = os.path.join(model_dir, 'solver.prototxt')

In [ ]:
def combined_roidb(imdb_names):
    def get_roidb(imdb_name):
        print imdb_name
        imdb = get_imdb(imdb_name)
        print 'Loaded dataset `{:s}` for training'.format(imdb.name)
        imdb.set_proposal_method(cfg.TRAIN.PROPOSAL_METHOD)
        print 'Set proposal method: {:s}'.format(cfg.TRAIN.PROPOSAL_METHOD)
        roidb = get_training_roidb(imdb)
        return roidb

    roidbs = [get_roidb(s) for s in imdb_names.split('+')]
    roidb = roidbs[0]
    if len(roidbs) > 1:
        for r in roidbs[1:]:
            roidb.extend(r)
        imdb = datasets.imdb.imdb(imdb_names)
    else:
        imdb = get_imdb(imdb_names)
    return imdb, roidb

In [ ]:
cfg_from_file('../experiments/cfgs/faster_rcnn_end2end_dstl.yml')
# cfg.PIXEL_MEANS = np.array([[[102.34775165, 93.19367343, 84.36433397]]])
np.random.seed(cfg.RNG_SEED)
caffe.set_random_seed(cfg.RNG_SEED)
imdb_train, roidb_train = combined_roidb('dstl'+appendix+'_train')
imdb_val, roidb_val = combined_roidb('dstl'+appendix+'_val')

In [ ]:
def draw_bboxes(blobs):
    
    gt_boxes, data, im_info = blobs.values()
    N_boxes = gt_boxes.shape[0]
    print N_boxes
    
    if N_boxes > 0:        
        fig, ax = plt.subplots(1, 1)
        im = (data[0, ...].transpose((1, 2, 0)) + cfg.PIXEL_MEANS).astype(np.uint8)
        plt.imshow(im)
        for gt_box in gt_boxes:
            if classes is None:
                class_name = class_names[int(gt_box[-1]) - 1]
            else:
                class_name = class_names[classes[int(gt_box[-1]) - 1]]
            ax.add_patch(
                plt.Rectangle((gt_box[0], gt_box[1]),
                      gt_box[2] - gt_box[0],
                      gt_box[3] - gt_box[1], fill=False,
                      edgecolor=get_color(class_name), linewidth=3)
            )
            ax.text(gt_box[0], gt_box[1], '{}'.format(class_name), color=get_color(class_name), fontsize=18, verticalalignment='bottom')
        plt.show()
        time.sleep(2.)
        

Retrieve the 'blobs' from the input-data layer

In [ ]:
for roidb_entry in roidb_val:
    print roidb_entry['image']
    # fetch multple batches/tiles per image,
    for i in range(2):
        blobs = minibatch.get_minibatch([roidb_entry], N_classes)
        draw_bboxes(blobs)

In [ ]:
blobs.keys()
print blobs['data'].shape
print blobs['gt_boxes'].shape
print blobs['gt_boxes'].min(axis=0)
print blobs['gt_boxes'].max(axis=0)
print blobs['im_info']

In [ ]:
cfg.TRAIN.SCALES

## 